
# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** I tried to adopt the architecture to "Show, Attend and Tell" paper: https://arxiv.org/pdf/1502.03044.pdf
As well, I followed very useful tutorial, provided some insights: https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning

 Model consists of 2 parts:
  - Encoder - CNN Resnet152, that outputs a feature maps of shape (batch, pixels, num_feature_maps). I did not use the fully-connected layer to make the RNN pay attention to a specific feature map at the Decoder stage.
  - Decoder - LSTMCell, empowered with Bahdanau attention. As well, I found it very helpful using "gated" attention, when we multiply the context vector by sigmoid of hidden units linear activation. This allows the model to focus on objects image (proposed by athours of paper). Further, I'd like to experiment with ReLU activation for "gaten" attention.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
- I've stayed with the same transformers, since the Random crop and image augmentation bring more variance to training data that helps to handle overfitting. Normalization usually brings stability during the training process. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** <br>
The only parameters I learned were *Decoder's parameters* since I removed the fully-connected layer of pre-trained CNN Encoder.<br>
As mentioned above, CNN Encoder (resnet152) produces the feature maps to make attention focus on specific image parts. Therefore, no need to fine-tune CNN.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** <br>
I used Adam optimizer since it works effectively with sparse gradient and posesses the properties of both AdaGrad and RMSProp. It is also a common choice, working with large datasets and highly parameterized models.<br>
Besides, I decreased the learning rate to `1e-4` in order to add a regularization effect for my complex model and avoid overshooting.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
import sys
import os
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import torch.utils.data as data
from utils import get_batch_caps, get_hypothesis

import nltk
from nltk.translate.bleu_score import corpus_bleu

nltk.download("punkt")

from pathlib import Path

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64  # batch size, change to 64
vocab_threshold = 3  # minimum word count threshold
vocab_from_file = True  # if True, load existing vocab file
embed_size = 256  # dimensionality of image and word embeddings
hidden_size = 512  # number of features in hidden state of the RNN decoder
num_features = 2048  # number of feature maps, produced by Encoder
num_epochs = 14  # number of training epochs
save_every = 1  # determines frequency of saving model weights
print_every = 100  # determines window for printing average loss
data_path = Path("data/train/")

log_train = "training_log.txt"  # name of files with saved training loss and perplexity
log_val = "validation_log.txt"
bleu = "bleu.txt"

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose(
    [
        transforms.Resize(256),  # smaller edge of image resized to 256
        transforms.RandomCrop(224),  # get 224x224 crop from random location
        transforms.RandomHorizontalFlip(),  # horizontally flip image with probability=0.5
        transforms.ToTensor(),  # convert the PIL Image to a tensor
        transforms.Normalize(
            (0.485, 0.456, 0.406),  # normalize image for pre-trained model
            (0.229, 0.224, 0.225),
        ),
    ]
)

# Build data loader.
data_loader = get_loader(
    data_path=data_path,
    transform=transform_train,
    mode="train",
    batch_size=batch_size,
    vocab_threshold=vocab_threshold,
    vocab_from_file=vocab_from_file,
)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder.
encoder = EncoderCNN()
decoder = DecoderRNN(
    num_features=num_features,
    embedding_dim=embed_size,
    hidden_dim=hidden_size,
    vocab_size=vocab_size,
)

# Move models to GPU if CUDA is available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)


# Define the loss function.
criterion = (
    nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()
)

# TODO #3: Specify the learnable parameters of the model.
# params = list(decoder.parameters()) + list(encoder.parameters())
params = list(decoder.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=1e-4)

# Set the total number of training steps per epoch.
total_step = math.ceil(
    len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size
)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Vocabulary successfully loaded from vocab.pkl file!


100%|██████████| 30000/30000 [00:05<00:00, 5366.02it/s]
c:\Users\Lenovo\miniconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\miniconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Set up the loader for *validation* dataset.

In [2]:
# Setup the transforms
transform_test = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # smaller edge of image resized to 256
        transforms.ToTensor(),  # convert the PIL Image to a tensor
        transforms.Normalize(
            (0.485, 0.456, 0.406),  # normalize image for pre-trained model
            (0.229, 0.224, 0.225),
        ),
    ]
)

# Create the data loader.
valid_data_loader = get_loader(
    data_path=data_path, transform=transform_test, batch_size=batch_size, mode="dev"
)

Vocabulary successfully loaded from vocab.pkl file!


100%|██████████| 5000/5000 [00:00<00:00, 6416.00it/s]


In [3]:
total_step_valid = math.ceil(
    len(valid_data_loader.dataset.caption_lengths)
    / valid_data_loader.batch_sampler.batch_size
)
total_step_valid

79

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [4]:
def train(
    epoch,
    encoder,
    decoder,
    optimizer,
    criterion,
    total_step,
    num_epochs,
    data_loader,
    write_file,
    save_every=1,
):
    """Train function for a single epoch.
    Arguments:
    ----------
    - epoch - number of current epoch
    - encoder - model's Encoder
    - decoder - model's Decoder
    - optimizer - model's optimizer (Adam in our case)
    - criterion - loss function to optimize
    - num_epochs - total number of epochs
    - data_loader - specified data loader (for training, validation or test)
    - write_file - file to write the training logs

    """
    epoch_loss = 0.0
    epoch_perplex = 0.0

    for i_step in range(1, total_step + 1):
        # training mode on
        encoder.eval()  # no fine-tuning for Encoder
        decoder.train()

        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        images, captions = next(iter(data_loader))
        # target captions, excluding the first word
        captions_target = captions[:, 1:].to(device)
        # captions for training without the last word
        captions_train = captions[:, :-1].to(device)

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)

        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()

        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs, atten_weights = decoder(captions=captions_train, features=features)

        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions_target.reshape(-1))

        # Backward pass.
        loss.backward()

        # Update the parameters in the optimizer.
        optimizer.step()

        perplex = np.exp(loss.item())
        epoch_loss += loss.item()
        epoch_perplex += perplex

        stats = (
            "Epoch train: [%d/%d], Step train: [%d/%d], Loss train: %.4f, Perplexity train: %5.4f"
            % (epoch, num_epochs, i_step, total_step, loss.item(), perplex)
        )

        # Print training statistics (on same line).
        print("\r" + stats, end="")
        sys.stdout.flush()

        # Print training statistics to file.
        write_file.write(stats + "\n")
        write_file.flush()

        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print("\r" + stats)

    epoch_loss_avg = epoch_loss / total_step
    epoch_perp_avg = epoch_perplex / total_step

    print("\r")
    print("Epoch train:", epoch)
    print(
        "\r"
        + "Avg. Loss train: %.4f, Avg. Perplexity train: %5.4f"
        % (epoch_loss_avg, epoch_perp_avg),
        end="",
    )
    print("\r")

    # Save the weights.
    if epoch % save_every == 0:
        torch.save(
            decoder.state_dict(), os.path.join("./models_new", "decoder-%d.pkl" % epoch)
        )
        torch.save(
            encoder.state_dict(), os.path.join("./models_new", "encoder-%d.pkl" % epoch)
        )

In [5]:
def validate(
    epoch,
    encoder,
    decoder,
    optimizer,
    criterion,
    total_step,
    num_epochs,
    data_loader,
    write_file,
    bleu_score_file,
):
    """Validation function for a single epoch.
    Arguments:
    ----------
    - epoch - number of current epoch
    - encoder - model's Encoder (evaluation)
    - decoder - model's Decoder (evaluation)
    - optimizer - model's optimizer (Adam in our case)
    - criterion - optimized loss function
    - num_epochs - total number of epochs
    - data_loader - specified data loader (for training, validation or test)
    - write_file - file to write the validation logs
    """
    epoch_loss = 0.0
    epoch_perplex = 0.0
    references = []
    hypothesis = []

    for i_step in range(1, total_step + 1):
        # evaluation of encoder and decoder
        encoder.eval()
        decoder.eval()
        val_images, val_captions, caps_all = next(iter(data_loader))

        val_captions_target = val_captions[:, 1:].to(device)
        val_captions = val_captions[:, :-1].to(device)
        val_images = val_images.to(device)

        features_val = encoder(val_images)
        outputs_val, atten_weights_val = decoder(
            captions=val_captions, features=features_val
        )
        loss_val = criterion(
            outputs_val.view(-1, vocab_size), val_captions_target.reshape(-1)
        )

        # preprocess captions and add them to the list
        caps_processed = get_batch_caps(caps_all, batch_size=batch_size)
        references.append(caps_processed)
        # get corresponding indicies from predictions
        # and form hypothesis from output
        terms_idx = torch.max(outputs_val, dim=2)[1]
        hyp_list = get_hypothesis(terms_idx, data_loader=data_loader)
        hypothesis.append(hyp_list)

        perplex = np.exp(loss_val.item())
        epoch_loss += loss_val.item()
        epoch_perplex += perplex

        stats = (
            "Epoch valid: [%d/%d], Step valid: [%d/%d], Loss valid: %.4f, Perplexity valid: %5.4f"
            % (epoch, num_epochs, i_step, total_step, loss_val.item(), perplex)
        )

        # Print training statistics (on same line).
        print("\r" + stats, end="")
        sys.stdout.flush()

        # Print training statistics to file.
        write_file.write(stats + "\n")
        write_file.flush()

        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print("\r" + stats)

    epoch_loss_avg = epoch_loss / total_step
    epoch_perp_avg = epoch_perplex / total_step

    # prepare the proper shape for computing BLEU scores
    references = np.array(references).reshape(total_step * batch_size, -1)
    # hyps = np.array(hypothesis).reshape(total_step*batch_size, -1)
    hyps = np.concatenate(np.array(hypothesis))

    bleu_1 = corpus_bleu(references, hyps, weights=(1.0, 0, 0, 0))
    bleu_2 = corpus_bleu(references, hyps, weights=(0.5, 0.5, 0, 0))
    bleu_3 = corpus_bleu(references, hyps, weights=(1.0 / 3.0, 1.0 / 3.0, 1.0 / 3.0, 0))
    bleu_4 = corpus_bleu(references, hyps, weights=(0.25, 0.25, 0.25, 0.25))
    # append individual n_gram scores
    # bleu_score_list.append((bleu_1, bleu_2, bleu_3, bleu_4))

    print("\r")
    print("Epoch valid:", epoch)
    epoch_stat = (
        "Avg. Loss valid: %.4f, Avg. Perplexity valid: %5.4f, \
    BLEU-1: %.2f, BLEU-2: %.2f, BLEU-3: %.2f, BLEU-4: %.2f"
        % (epoch_loss_avg, epoch_perp_avg, bleu_1, bleu_2, bleu_3, bleu_4)
    )

    print("\r" + epoch_stat, end="")
    print("\r")

    bleu_score_file.write(epoch_stat + "\n")
    bleu_score_file.flush()

In [6]:
# Open the training log file.
file_train = open(log_train, "w")
file_val = open(log_val, "w")
bleu_score_file = open(bleu, "w")


# store BLEU scores in list
bleu_scores = []
total_step_valid = math.ceil(
    len(valid_data_loader.dataset.caption_lengths)
    / valid_data_loader.batch_sampler.batch_size
)

for epoch in range(0, num_epochs + 1):
    train(
        epoch,
        encoder,
        decoder,
        optimizer,
        criterion,
        total_step,
        num_epochs=num_epochs,
        data_loader=data_loader,
        write_file=file_train,
        save_every=1,
    )

    validate(
        epoch,
        encoder,
        decoder,
        optimizer,
        criterion,
        total_step=total_step_valid,
        num_epochs=num_epochs,
        data_loader=valid_data_loader,
        write_file=file_val,
        bleu_score_file=bleu_score_file,
    )

file_train.close()
file_val.close()
bleu_score_file.close()

Epoch train: [0/14], Step train: [2/469], Loss train: 3.7883, Perplexity train: 44.1819

KeyboardInterrupt: 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.

## Testing stage

In [ ]:
# Setup the transforms
transform_test = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # smaller edge of image resized to 256
        transforms.ToTensor(),  # convert the PIL Image to a tensor
        transforms.Normalize(
            (0.485, 0.456, 0.406),  # normalize image for pre-trained model
            (0.229, 0.224, 0.225),
        ),
    ]
)

In [ ]:
test_data_loader = get_loader(transform=transform_test, batch_size=1, mode="test")

In [ ]:
vocab_size = len(test_data_loader.dataset.vocab)
vocab_size

In [ ]:
encoder_file = "encoder-14.pkl"
decoder_file = "decoder-14.pkl"


embed_size = 256  # dimensionality of image and word embeddings
hidden_size = 512  # number of features in hidden state of the RNN decoder
num_features = 2048  # number of feature maps, produced by Encoder

# The size of the vocabulary.
vocab_size = len(test_data_loader.dataset.vocab)

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN()
encoder.eval()
decoder = DecoderRNN(
    num_features=num_features,
    embedding_dim=embed_size,
    hidden_dim=hidden_size,
    vocab_size=vocab_size,
)
decoder.eval()


# Load the trained weights.
encoder.load_state_dict(
    torch.load(os.path.join("./models_new", encoder_file), map_location="cpu")
)
decoder.load_state_dict(
    torch.load(os.path.join("./models_new", decoder_file), map_location="cpu")
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

### Image example

In [ ]:
orig_image, image = next(iter(test_data_loader))

In [ ]:
print("Image shape:", list(image.size()))
print("Original image shape:", list(orig_image.size()))

Define functions for sampling and predicting.

In [ ]:
def clean_sentence(output, data_loader):
    vocab = data_loader.dataset.vocab.idx2word
    words = [vocab.get(idx) for idx in output]
    words = [word for word in words if word not in (",", ".", "<end>")]
    sentence = " ".join(words)

    return sentence

In [ ]:
def get_prediction(data_loader):
    orig_image, image = next(iter(data_loader))
    plt.imshow(orig_image.squeeze())
    plt.title("Sample Image")
    plt.show()
    image = image.to(device)
    features = encoder(image)
    output = decoder.sample(features)
    sentence = clean_sentence(output)
    print(sentence)

In [ ]:
# Visualize sample image, before pre-processing.
plt.imshow(np.squeeze(orig_image))
plt.title("example image")
plt.show()

In [ ]:
image = image.to(device)

# Obtain the embedded image features.
print(image.shape)
features = encoder(image)

# Pass the embedded image features through the model to get a predicted caption.
output = decoder.sample(features)
print("example output:", output)
assert type(output) == list, "Output needs to be a Python list"
assert all([type(x) == int for x in output]), "Output should be a list of integers."
assert all(
    [x in test_data_loader.dataset.vocab.idx2word for x in output]
), "Each entry in the output needs to correspond to an integer that indicates a token in the vocabulary."

In [ ]:
sentence = clean_sentence(output, test_data_loader)
print("example sentence:", sentence)